In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import torch
import math
from transformers import AutoTokenizer, AutoModelForSequenceClassification
!pip install -qqq transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00


In [ ]:
# Change to your own directory
try:
    os.chdir("/content/drive/MyDrive/electra/")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

Directory changed


In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import time

# Initialize Firebase Admin SDK
cred = credentials.Certificate("journal-app-bd481-firebase-adminsdk-26qt3-ae08cf2035.json")
firebase_admin.initialize_app(cred)

# Initialize Firestore client
db = firestore.client()



In [ ]:
import torch.nn.functional as F
suicide_tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")
suicide_model =  AutoModelForSequenceClassification.from_pretrained("./")
def check_intent(text):
    """Check if suicidal intent is present in text"""
    global suicide_tokenizer, suicide_model

    # Tokenize the input text
    tokenized_text = suicide_tokenizer.encode_plus(text, return_tensors="pt")

    # Get the model predictions
    logits = suicide_model(**tokenized_text)[0]

    # Apply sigmoid activation to convert logits to probabilities
    probabilities = torch.sigmoid(logits)

    # Extract probability of positive class (indicating suicidal intent)
    suicidal_intent_probability = probabilities[0][1].item() * 100

    return round(suicidal_intent_probability, 2)

check_intent("I had a bad day.Everyone is so mean to me.I dont know what to do")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

40.29

In [ ]:
check_intent("")

92.95

MODIFIED

Final Server


In [ ]:
def on_snapshot(col_snapshot, changes, read_time):
    for change in changes:
        if change.type.name == 'MODIFIED':
            print(f"Document {change.document.id} has been modified.")
            # Get the document ID
            document_id = change.document.id
            print("Document ID:", document_id)

            # Reference to the document inside the "class" collection
            class_doc_ref = db.collection('class').document(document_id)

            # Get the snapshot of the document
            class_doc_snapshot = class_doc_ref.get()

            # Check if the document exists and contains the "verification" field
            if class_doc_snapshot.exists:
                class_data = class_doc_snapshot.to_dict()
                verification_status = class_data.get('verification')

                # Check if the "verification" field is 0
                if verification_status == 0:
                  print("Verification status is 0 for document:", document_id)

                    # Perform your desired operation here

                  doc_ref = db.collection('diary').document(document_id)

                   # Get the snapshot of the document's collections
                  collections = doc_ref.collections()

                  # Iterate through each collection inside the document
                  for collection_ref in collections:
                  # Get the collection ID (which represents user IDs in this case)
                    user_id = collection_ref.id

                  # Reference to the document inside the user ID collection ('verification' subcollection)
                    user_verification_doc_ref = collection_ref.document('verification')

                  # Get the snapshot of the document inside the user ID collection
                    user_verification_doc_snapshot = user_verification_doc_ref.get()






                  # Check if the document inside the user ID collection exists
                    if user_verification_doc_snapshot.exists:
                      # Get the data of the document inside the user ID collection
                      user_data = user_verification_doc_snapshot.to_dict()

                      # Check the value of the 'ver_stat' field
                      ver_stat = user_data.get('verification')

                      # Check if 'ver_stat' field exists and its value indicates verification
                      if ver_stat == 0:
                          #Reference to the "recent" document inside the user ID collection
                          recent_doc_ref = collection_ref.document('recent')
                          #class_ref=db.collection('class').document(document_id)
                          recent_ver_ref =collection_ref.document('verification')
                          recent_ver_ref.update({'verification':1})

                          # Get the snapshot of the "recent" document
                          recent_doc_snapshot = recent_doc_ref.get()

                          # Check if the "recent" document exists
                          if recent_doc_snapshot.exists:
                              recent_data = recent_doc_snapshot.to_dict()
                              recent_post = recent_data.get('recent')

                              #MODEL
                              suicide_tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")
                              suicide_model =  AutoModelForSequenceClassification.from_pretrained("./")
                              def check_intent(text):
                                """Check if suicidal intent is present in text"""
                                global suicide_tokenizer, suicide_model

                                # Tokenize the input text
                                tokenized_text = suicide_tokenizer.encode_plus(text, return_tensors="pt")

                                # Get the model predictions
                                logits = suicide_model(**tokenized_text)[0]

                                # Apply sigmoid activation to convert logits to probabilities
                                probabilities = torch.sigmoid(logits)

                                 # Extract probability of positive class (indicating suicidal intent)
                                suicidal_intent_probability = probabilities[0][1].item() * 100

                                return math.floor(suicidal_intent_probability)

                              percent = check_intent(recent_post)



                              # Update the "write_entry" field in the "model" collection
                              model_doc_ref = db.collection('model').document('model').collection(document_id).document(user_id)
                              model_doc_ref.set({'recent': recent_post,'percentage' : percent}, merge=True)

                              print("Updated 'write_entry' for user ID:", user_id)
                          else:
                              print("No 'recent' document found for user ID:", user_id)
                      else:
                          print("User ID is not verified:", user_id)
                    else:
                      print("No 'verification' document found for user ID:", user_id)

                    class_doc_ref.update({'verification': 1})  # Update verification status

                    # Process recent post
                    # You can add your existing code here for processing recent post
                    # Example: recent_doc_ref = class_doc_ref.collection('recent').document('recent')
                    # ...

                else:
                    print("Verification status is not 0 for document:", document_id)
            else:
                print("Document does not exist:", document_id)


# Define the reference to the Firestore collection to listen for changes
collection_ref = db.collection("class")

# Create a listener to monitor changes in the collection
doc_watch = collection_ref.on_snapshot(on_snapshot)

try:
    while True:
        # Keep the script running indefinitely
        time.sleep(1)

except KeyboardInterrupt:
    # Stop the listener when the program is interrupted
    doc_watch.unsubscribe()


Document cse beta has been modified.
Document ID: cse beta
Verification status is 0 for document: cse beta
User ID is not verified: u2003075
User ID is not verified: u2003076
Updated 'write_entry' for user ID: u2003130
Document cse beta has been modified.
Document ID: cse beta
Verification status is not 0 for document: cse beta
Document cse beta has been modified.
Document ID: cse beta
Verification status is 0 for document: cse beta
User ID is not verified: u2003075
User ID is not verified: u2003076
Updated 'write_entry' for user ID: u2003130
Document cse beta has been modified.
Document ID: cse beta
Verification status is not 0 for document: cse beta
Document cse beta has been modified.
Document ID: cse beta
Verification status is 0 for document: cse beta
User ID is not verified: u2003075
User ID is not verified: u2003076
Updated 'write_entry' for user ID: u2003130
Document cse beta has been modified.
Document ID: cse beta
Verification status is not 0 for document: cse beta


Test

In [ ]:
# Reference to the document containing the collection of user IDs
doc_ref = db.collection('diary').document('cse beta')

# Get the snapshot of the document's collections
collections = doc_ref.collections()

# Iterate through each collection inside the document
for collection_ref in collections:
    # Get the collection ID (which represents user IDs in this case)
    user_id = collection_ref.id

    # Reference to the document inside the user ID collection ('verification' subcollection)
    user_verification_doc_ref = collection_ref.document('verification')

    # Get the snapshot of the document inside the user ID collection
    user_verification_doc_snapshot = user_verification_doc_ref.get()






    # Check if the document inside the user ID collection exists
    if user_verification_doc_snapshot.exists:
        # Get the data of the document inside the user ID collection
        user_data = user_verification_doc_snapshot.to_dict()

        # Check the value of the 'ver_stat' field
        ver_stat = user_data.get('verification')

        # Check if 'ver_stat' field exists and its value indicates verification
        if ver_stat == 0:
            # Reference to the "recent" document inside the user ID collection
            recent_doc_ref = collection_ref.document('recent')

            # Get the snapshot of the "recent" document
            recent_doc_snapshot = recent_doc_ref.get()

            # Check if the "recent" document exists
            if recent_doc_snapshot.exists:
                recent_data = recent_doc_snapshot.to_dict()
                recent_post = recent_data.get('recent')

                #MODEL
                suicide_tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")
                suicide_model =  AutoModelForSequenceClassification.from_pretrained("./")
                def check_intent(text):
                  global suicide_tokenizer, suicide_model
                  tokenised_text = suicide_tokenizer.encode_plus(text, return_tensors="pt")

                  logits = suicide_model(**tokenised_text)[0]

                  prediction = torch.softmax(logits, dim=1).tolist()[0][1]

                  return round(prediction*100,2)

                percent = check_intent(recent_post)



                # Update the "write_entry" field in the "model" collection
                model_doc_ref = db.collection('model').document("cse beta").collection(user_id).document(user_id)
                model_doc_ref.set({'recent': recent_post,'percentage' : percent}, merge=True)

                print("Updated 'write_entry' for user ID:", user_id)
            else:
                print("No 'recent' document found for user ID:", user_id)
        else:
            print("User ID is not verified:", user_id)
    else:
        print("No 'verification' document found for user ID:", user_id)



User ID is not verified: u2003000
Updated 'write_entry' for user ID: u2003075
User ID is not verified: u2003130


In [ ]:
suicide_tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")
suicide_model =  AutoModelForSequenceClassification.from_pretrained("./")
def check_intent(text):
  """Check if suicidal intent is present in text"""

  global suicide_tokenizer, suicide_model

  tokenised_text = suicide_tokenizer.encode_plus(text, return_tensors="pt")

  logits = suicide_model(**tokenised_text)[0]

  prediction = torch.softmax(logits, dim=1).tolist()[0][1]
  return round(prediction*100,2)

check_intent("i dont want to live")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

99.48